<a href="https://colab.research.google.com/github/ichekhovskikh/recommendation-system/blob/master/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import gensim
import os
import collections
import smart_open
import random
import json
import urllib.request
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from pymystem3 import Mystem

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 70 текстов.

И мы проверим нашу модель на глаз, используя тестовый корпус, который содержит 7 документов.

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать)

In [0]:
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/train.json'
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/test.json'

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

Лемматизация каждого слова статьи:

In [0]:
def lemmatize(text):
    return Mystem().lemmatize(text)

Удаление стоп слов:

In [0]:
def remove_stopwords(text):
    russian_stopwords = stopwords.words("russian")
    return [word for word in text if word not in russian_stopwords]

Отрытие файла с корпусом статей:

In [0]:
def read_corpus(corpus_path, tokens_only=False):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for article in corpus:
            normalized_text = gensim.utils.simple_preprocess(article['text'])
#             normalized_text = lemmatize(normalized_text)
            normalized_text = remove_stopwords(normalized_text)
            if tokens_only:
                yield normalized_text
            else:
                yield gensim.models.doc2vec.TaggedDocument(normalized_text, article['tags'])

Получение исходного текста статьи:

In [0]:
def get_article_text(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

In [0]:
train_corpus = list(read_corpus(train_path))
test_corpus = list(read_corpus(test_path, tokens_only=True))

Давайте посмотрим на учебный корпус:

In [56]:
print(train_corpus[:2])

[TaggedDocument(words=['относится', 'семье', 'языков', 'подобным', 'синтаксисом', 'синтаксис', 'наиболее', 'близок', 'java', 'язык', 'имеет', 'статическую', 'типизацию', 'поддерживает', 'полиморфизм', 'перегрузку', 'операторов', 'числе', 'операторов', 'явного', 'неявного', 'приведения', 'типа', 'делегаты', 'атрибуты', 'события', 'свойства', 'обобщённые', 'типы', 'методы', 'итераторы', 'анонимные', 'функции', 'поддержкой', 'замыканий', 'linq', 'исключения', 'комментарии', 'формате', 'xml', 'переняв', 'многое', 'своих', 'языков', 'pascal', 'модула', 'smalltalk', 'особенности', 'java', 'опираясь', 'практику', 'использования', 'исключает', 'некоторые', 'модели', 'проблематичные', 'разработке', 'программных', 'систем', 'например', 'отличие', 'некоторых', 'других', 'языков', 'поддерживает', 'множественное', 'наследование', 'классов', 'допускается', 'множественное', 'наследование', 'интерфейсов'], tags=['0']), TaggedDocument(words=['программы', 'java', 'транслируются', 'байт', 'код', 'java', 

Корпус тестирования выглядит так:

In [57]:
print(test_corpus[:2])

[['язык', 'ассемблера', 'машинно', 'ориентированный', 'язык', 'низкого', 'уровня', 'команды', 'прямо', 'соответствуют', 'отдельным', 'командам', 'машины', 'также', 'предоставлять', 'дополнительные', 'возможности', 'облегчения', 'такие', 'макрокоманды', 'выражения', 'средства', 'обеспечения', 'модульности', 'программ', 'рассматриваться', 'автокод', 'расширенный', 'конструкциями', 'языков', 'высокого', 'уровня', 'является', 'существенно', 'платформо', 'зависимым', 'языки', 'ассемблера', 'различных', 'аппаратных', 'платформ', 'несовместимы', 'хотя', 'могут', 'целом', 'подобны', 'русском', 'языке', 'именоваться', 'просто', 'ассемблером', 'типичны', 'выражения', 'типа', 'писать', 'программу', 'ассемблере', 'строго', 'говоря', 'неверно', 'ассемблером', 'именуется', 'утилита', 'трансляции', 'программы', 'языка', 'ассемблера', 'объектный', 'код', 'компьютера'], ['перитонит', 'развивается', 'вследствие', 'бактериального', 'инфицирования', 'брюшной', 'полости', 'подавляющего', 'большинства', 'па

Обратите внимание, что корпус тестирования представляет собой просто список списков и не содержит никаких тегов.

# Обучение модели
## Создание объекта Doc2Vec
Теперь мы создадим модель Doc2Vec с векторным размером 80 слов и перебираем учебный корпус 100 раз. Мы устанавливаем минимальную длину слова равной двум, чтобы отбрасывать слова с очень малым количеством вхождений.

Однако это очень маленький набор данных (70 документов) с короткими текстами (несколько сотен слов). 

Словарь содержит в себе все уникальные слова, извлеченных из учебного корпуса.

In [58]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=80, min_count=2, epochs=100)
model.build_vocab(train_corpus)
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 359 ms, sys: 27.9 ms, total: 387 ms
Wall time: 388 ms


## Inferring a Vector
Важно отметить, что теперь вы можете вывести вектор для любого фрагмента текста без необходимости переобучать модель, передав список слов в функцию model.infer_vector. Затем этот вектор можно сравнить с другими векторами по косинусной близости.

In [59]:
text = 'Горностай небольшой зверек семейства куньих очень ценится'
# model.infer_vector(lemmatize(text))
model.infer_vector(text.split())

array([ 1.02238491e-01,  1.28838792e-01, -8.24554339e-02,  1.09257273e-01,
        9.15161669e-02, -3.61136929e-03,  1.44914851e-01, -1.07948072e-01,
        8.57756287e-02,  6.51605427e-02,  4.11159955e-02,  9.15924758e-02,
       -1.62360057e-01, -8.20722617e-03, -6.05938844e-02,  3.34975682e-02,
       -2.52091866e-02, -5.64201921e-02, -2.29353145e-01,  2.98975222e-02,
       -3.31879295e-02,  8.43746364e-02,  1.75659508e-01, -9.72581059e-02,
       -1.17370501e-01,  3.05420998e-02, -7.89900422e-02, -1.57926902e-01,
        1.48934558e-01,  1.41719915e-02,  7.50204623e-02,  3.47862616e-02,
       -2.14287569e-03,  5.63786775e-02, -1.18121449e-02,  5.13272267e-03,
        1.19865291e-01,  4.12705950e-02, -9.19697285e-02,  6.78503141e-02,
        4.23179604e-02, -3.42725441e-02, -4.85965656e-03,  8.71589780e-02,
       -1.43224164e-03,  3.87515314e-02,  8.11177865e-02,  2.71681063e-02,
        9.48282145e-03, -1.01373410e-02,  1.27145246e-01, -1.54269040e-01,
        9.79013625e-04,  

Обратите внимание, что, поскольку лежащие в основе алгоритмы обучения представляют собой подходы на основе итеративной аппроксимации, в которой используется внутренняя рандомизация, таким обрахрм, повторные выводы одного и того же текста будут возвращать слегка разные векторы.

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

Проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя адекватно, хотя и не является реальным значением «точности».

Можем взглянуть на пример:

In [78]:
doc_id = random.randint(0, len(train_corpus) - 1)

inferred_vector = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text(doc_id, train_path)))
for label, index in [('1)', 0), ('2', 1), ('3)', 2), ('4)', 3)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], get_article_text(int(sims[index][0]), train_path)))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


ТЕКСТ ИСХДНОГО ДОКУМЕНТА (2): «C++ широко используется для разработки программного обеспечения, являясь одним из самых популярных языков программирования. Область его применения включает создание операционных систем, разнообразных прикладных программ, драйверов устройств, приложений для встраиваемых систем, высокопроизводительных серверов, а также развлекательных приложений (игр). Существует множество реализаций языка C++, как бесплатных, так и коммерческих и для различных платформ. Например, на платформе x86 это GCC, Visual C++, Intel C++ Compiler, Embarcadero (Borland) C++ Builder и другие. C++ оказал огромное влияние на другие языки программирования, в первую очередь на Java и C#. Синтаксис C++ унаследован от языка C. Одним из принципов разработки было сохранение совместимости с C. Тем не менее, C++ не является в строгом смысле надмножеством C; множество программ, которые могут одинаково успешно транслироваться как компиляторами C, так и компиляторами C++, довольно велико, но не вкл

Обратите внимание, что наиболее похожий документ (как правило, тот же текст) имеет параметр сходства, приближающийся к единице. 

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [83]:
doc_id = random.randint(0, len(test_corpus) - 1)

inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text(doc_id, test_path)))
for label, index in [('1)', 0), ('2', 1), ('3)', 2), ('4)', 3)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], get_article_text(int(sims[index][0]), train_path)))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


ТЕКСТ ИСХДНОГО ДОКУМЕНТА (5): «Апельсин - постоянно зелёное фруктово-плодовое древо семейства апельсин, рода рутовых. В одичавшем варианте никогда не попадается, всегда только плодоносящее вкусными апельсинами. В сильнорослых местах оно доходит вышины 12 метров в высоту, в самых же миниатюрных 4-6 метров. Листья у него очень кожистые, слегка округлые, с заострённой на концах вершиной. Цветочки же апельсинового дерева двуполые, белоснежные, очень пахучие, единичные либо в соцветиях. Результаты - многогнёздная ягодка; в подчиненности от их вида они теперь очень отличаются, как по своему объему так и своей окраске кожуры апельсина. Внутренность насыщенная, сладостная либо приторно-сладостная. В протяжение вегетационного этапа у дерева апельсина в связи с погодными обстоятельствами способны являться даже несколько этапов увеличения их, любой с которых заменяется этапом спокойствия. В присутствии подходящих обстоятельств, способен существовать и приносить свои вкусные плоды может наиболее 7